Step 1: Import required libraries and load preprocessed EEG data
----------------------------------------------------------------
In this step, we import all necessary Python packages for EEG data analysis and machine learning.
We then load the preprocessed EEG data from the ADHD and Control groups.
The data is organized into a DataFrame with subject ID, class label (ADHD or Control),
and EEG channel data.

In [1]:
import numpy as np
import pandas as pd
import mne
from scipy import signal
import matplotlib.pyplot as plt
import os
from glob import glob
from scipy.io import loadmat
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
# Define paths to preprocessed data folders
data_root = 'ADHD_data'
adhd_folders = [
    os.path.join(data_root, 'ADHD_part1_preprocessed'),
    os.path.join(data_root, 'ADHD_part2_preprocessed')
]
control_folders = [
    os.path.join(data_root, 'Control_part1_preprocessed'),
    os.path.join(data_root, 'Control_part2_preprocessed')
]

# Function to load all preprocessed .mat files and create a DataFrame
def load_preprocessed_data(folders, class_label):
    all_data = []
    
    for folder in folders:
        mat_files = glob(os.path.join(folder, '*_preprocessed.mat'))
        
        for mat_file in mat_files:
            # Extract subject ID from filename
            subject_id = os.path.basename(mat_file).split('_')[0]
            
            # Load .mat file
            data = loadmat(mat_file)
            eeg_data = data['preprocessedData']  # Shape: (channels, time_points)
            fs = float(data['fs'][0, 0])  # Sampling frequency
            
            # Create a dictionary with subject info and EEG data
            subject_dict = {
                'ID': subject_id,
                'Class': class_label,
                'EEG_Data': eeg_data,
                'fs': fs
            }
            
            all_data.append(subject_dict)
    
    return all_data

In [3]:
# Load ADHD and Control data
print("Loading ADHD data...")
adhd_data = load_preprocessed_data(adhd_folders, 'ADHD')
print(f"Loaded {len(adhd_data)} ADHD subjects")

print("Loading Control data...")
control_data = load_preprocessed_data(control_folders, 'Control')
print(f"Loaded {len(control_data)} Control subjects")

# Combine all data
all_subjects = adhd_data + control_data
print(f"Total subjects: {len(all_subjects)}")

# Create a DataFrame
data = pd.DataFrame(all_subjects)
print(f"Data shape: {data.shape}")
print(data[['ID', 'Class']].head())

# Check sampling rate
sfreq = data['fs'].iloc[0]
print(f"Sampling frequency: {sfreq} Hz")


Loading ADHD data...
Loaded 61 ADHD subjects
Loading Control data...
Loaded 60 Control subjects
Total subjects: 121
Data shape: (121, 4)
     ID Class
0  v10p  ADHD
1  v12p  ADHD
2  v14p  ADHD
3  v15p  ADHD
4  v173  ADHD
Sampling frequency: 128.0 Hz


Step 2: Define frequency bands and create a function to calculate band power
---------------------------------------------------------------------------
Here we define common EEG frequency bands (delta, theta, alpha, beta) and create
a function to calculate the power in each frequency band for a given EEG signal.
The function applies a bandpass filter to the signal and then calculates the
mean of the squared signal as a measure of power.

In [4]:
# Define frequency bands
freq_bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30)
}

def calculate_band_power(data, sf, band):
    """
    Calculate the power of a specific frequency band in an EEG signal.
    
    Parameters:
    -----------
    data : 1D array
        EEG signal time series
    sf : float
        Sampling frequency in Hz
    band : tuple
        Frequency band range (low_freq, high_freq) in Hz
    
    Returns:
    --------
    float
        Power in the specified frequency band
    """
    # Get band frequencies
    low_freq, high_freq = band
    
    # Apply bandpass filter using MNE
    filtered_data = mne.filter.filter_data(
        data.astype(np.float64),  # Convert to float64 for better precision
        sfreq=sf,
        l_freq=low_freq,
        h_freq=high_freq,
        method='fir',
        verbose=False
    )
    
    # Calculate power (mean of squared signal)
    power = np.mean(filtered_data ** 2)
    
    return power

Step 3: Extract spectral power features for each subject
--------------------------------------------------------
For each subject, we extract the EEG data for all channels and calculate
the power in each frequency band for each channel. We also calculate the
Theta/Beta ratio for frontal channels, which is a common biomarker for ADHD.
All these features are collected into a feature matrix X_band.

In [5]:
# Define frontal channels for Theta/Beta ratio
frontal_channels = [0, 1, 2, 3, 4, 5, 16]  # Assuming Fp1, Fp2, F3, F4, F7, F8, Fz

# Initialize lists to store features and labels
band_powers = []
subject_ids = []
labels = []

# Process each subject
for idx, subject in data.iterrows():
    # Get subject info
    subject_id = subject['ID']
    subject_class = subject['Class']
    eeg_data = subject['EEG_Data']  # Shape: (channels, time_points)
    sf = subject['fs']
    
    # Initialize feature vector for this subject
    subject_features = []
    
    # Calculate band power for each channel and frequency band
    num_channels = eeg_data.shape[0]
    
    for ch in range(num_channels):
        channel_data = eeg_data[ch, :]
        
        # Skip channels with NaN values
        if np.isnan(channel_data).any():
            # Add zeros for this channel's features
            subject_features.extend([0, 0, 0, 0])
            continue
        
        # Calculate power for each frequency band
        for band_name, band_range in freq_bands.items():
            power = calculate_band_power(channel_data, sf, band_range)
            subject_features.append(power)
    
    # Calculate Theta/Beta ratio for frontal channels
    theta_powers = []
    beta_powers = []
    
    for ch_idx in frontal_channels:
        if ch_idx < num_channels:  # Make sure channel exists
            channel_data = eeg_data[ch_idx, :]
            
            # Skip channels with NaN values
            if np.isnan(channel_data).any():
                continue
            
            theta_power = calculate_band_power(channel_data, sf, freq_bands['theta'])
            beta_power = calculate_band_power(channel_data, sf, freq_bands['beta'])
            
            if beta_power > 0:  # Avoid division by zero
                theta_powers.append(theta_power)
                beta_powers.append(beta_power)
    
    # Calculate average Theta/Beta ratio across frontal channels
    if len(beta_powers) > 0:
        avg_theta = np.mean(theta_powers)
        avg_beta = np.mean(beta_powers)
        theta_beta_ratio = avg_theta / avg_beta
    else:
        theta_beta_ratio = 0
    
    # Add Theta/Beta ratio to features
    subject_features.append(theta_beta_ratio)
    
    # Add features to the list
    band_powers.append(subject_features)
    subject_ids.append(subject_id)
    labels.append(1 if subject_class == 'ADHD' else 0)
    
    print(f"Processed subject {subject_id} ({subject_class}): {len(subject_features)} features")

Processed subject v10p (ADHD): 77 features
Processed subject v12p (ADHD): 77 features
Processed subject v14p (ADHD): 77 features
Processed subject v15p (ADHD): 77 features
Processed subject v173 (ADHD): 77 features
Processed subject v18p (ADHD): 77 features
Processed subject v19p (ADHD): 77 features
Processed subject v1p (ADHD): 77 features
Processed subject v20p (ADHD): 77 features
Processed subject v21p (ADHD): 77 features
Processed subject v22p (ADHD): 77 features
Processed subject v24p (ADHD): 77 features
Processed subject v25p (ADHD): 77 features
Processed subject v27p (ADHD): 77 features
Processed subject v28p (ADHD): 77 features
Processed subject v29p (ADHD): 77 features
Processed subject v30p (ADHD): 77 features
Processed subject v31p (ADHD): 77 features
Processed subject v32p (ADHD): 77 features
Processed subject v33p (ADHD): 77 features
Processed subject v34p (ADHD): 77 features
Processed subject v35p (ADHD): 77 features
Processed subject v36p (ADHD): 77 features
Processed su

In [6]:
# Convert to numpy arrays
X_band = np.array(band_powers)
y_band = np.array(labels)

print(f"Feature matrix shape: {X_band.shape}")
print(f"Label vector shape: {y_band.shape}")

Feature matrix shape: (121, 77)
Label vector shape: (121,)


Step 4: Prepare labels and ensure consistent ordering
----------------------------------------------------
We ensure that the labels (y_band) are correctly aligned with the feature matrix (X_band)
by maintaining the same order of subject IDs.

In [7]:
# Create a DataFrame to verify alignment
feature_df = pd.DataFrame({
    'ID': subject_ids,
    'Class': y_band
})

print("Sample of subjects and their labels:")
print(feature_df.head())

# Check class balance
class_counts = feature_df['Class'].value_counts()
print("\nClass distribution:")
print(f"ADHD (1): {class_counts.get(1, 0)}")
print(f"Control (0): {class_counts.get(0, 0)}")

Sample of subjects and their labels:
     ID  Class
0  v10p      1
1  v12p      1
2  v14p      1
3  v15p      1
4  v173      1

Class distribution:
ADHD (1): 61
Control (0): 60


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X_band, y_band, test_size=0.3, random_state=42, stratify=y_band
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training set: 84 samples
Testing set: 37 samples


In [9]:
# Create a pipeline with StandardScaler and SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(probability=True))
])

# Define parameter grid for GridSearchCV
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01],
    'svm__kernel': ['linear', 'rbf']
}

# Create GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1,
    scoring='accuracy'
)

In [10]:
# Train the model
print("Training SVM with GridSearchCV...")
grid_search.fit(X_train, y_train)

# Get best model
best_model = grid_search.best_estimator_


Training SVM with GridSearchCV...
Fitting 5 folds for each of 32 candidates, totalling 160 fits


We evaluate the performance of the best model on the test set
and analyze the results using accuracy, confusion matrix, and classification report.

In [11]:
# Print best parameters
print("\nBest parameters:")
print(grid_search.best_params_)

# Evaluate on test set
test_accuracy = best_model.score(X_test, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")

# Make predictions
y_pred = best_model.predict(X_test)

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, y_pred, target_names=['Control', 'ADHD'])
print("\nClassification Report:")
print(class_report)

# Calculate and print additional metrics
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"\nSensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")

# Analyze feature importance (for linear kernel)
if 'linear' in grid_search.best_params_['svm__kernel']:
    # Get feature importance from SVM coefficients
    feature_importance = np.abs(best_model.named_steps['svm'].coef_[0])
    
    # Get top 10 features
    top_indices = np.argsort(feature_importance)[-10:]
    top_importance = feature_importance[top_indices]
    
    print("\nTop 10 most important features:")
    for i, idx in enumerate(top_indices):
        print(f"Feature {idx}: {top_importance[i]:.4f}")


Best parameters:
{'svm__C': 10, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}

Test accuracy: 0.4595

Confusion Matrix:
[[ 7 11]
 [ 9 10]]

Classification Report:
              precision    recall  f1-score   support

     Control       0.44      0.39      0.41        18
        ADHD       0.48      0.53      0.50        19

    accuracy                           0.46        37
   macro avg       0.46      0.46      0.46        37
weighted avg       0.46      0.46      0.46        37


Sensitivity (True Positive Rate): 0.5263
Specificity (True Negative Rate): 0.3889
